In [1]:
https://stackoverflow.com/questions/29528628/how-to-specify-a-variable-in-pandas-as-ordinal-categorical
# What this says is use factorize to convert discrete to categoricals.
# But use the custom mapper to convert ordinal to categorical, preserving the order
# So basically everything gets converted to numeric, but we factorize discrete, and ordinal, 
# we makes dummies from nominal

SyntaxError: invalid syntax (<ipython-input-1-85595b5a23a9>, line 1)

In [ ]:
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10, 8
rcParams.update({'font.size': 12})

In [ ]:
#Changing discrete to categorical
#Reading Data

housing_df = pd.read_excel('Data/AmesHousing.xls', delimiter=r"\s+")
housing_df['Order'] = hous_df['Order'].astype('object')
housing_df['Year Built'] = hous_df['Year Built'].astype('object')
housing_df['Year Remod/Add'] = hous_df['Year Remod/Add'].astype('object')
housing_df['Bsmt Full Bath'] = hous_df['Bsmt Full Bath'].astype('object')
housing_df['Bsmt Half Bath'] = hous_df['Bsmt Half Bath'].astype('object')
housing_df['Full Bath'] = hous_df['Full Bath'].astype('object')
housing_df['Half Bath'] = hous_df['Half Bath'].astype('object')
housing_df['Bedroom AbvGr'] = hous_df['Bedroom AbvGr'].astype('object')
housing_df['Kitchen AbvGr'] = hous_df['Kitchen AbvGr'].astype('object')
housing_df['TotRms AbvGrd'] = hous_df['TotRms AbvGrd'].astype('object')
housing_df['Fireplaces'] = hous_df['Fireplaces'].astype('object')
housing_df['Garage Yr Blt'] = hous_df['Garage Yr Blt'].astype('object')
housing_df['Garage Cars'] = hous_df['Garage Cars'].astype('object')
housing_df['Mo Sold'] = hous_df['Mo Sold'].astype('object')
housing_df['Yr Sold'] = hous_df['Yr Sold'].astype('object')



In [ ]:
housing_df[:5]

In [ ]:
housing_df['SalePrice'].hist(bins=50, alpha=0.8)
plt.axvline(housing_df['SalePrice'].mean(), color='r', linestyle='dashed', linewidth=2, label='Mean')
plt.axvline(housing_df['SalePrice'].median(), color='orange', linestyle='dashed', linewidth=2, label='Median')
plt.xlabel('SalePrice')
plt.ylabel('Count')
plt.legend()
plt.show()

In [ ]:
sns.boxplot(housing_df['SalePrice'], orient='h')
plt.show()

In [ ]:
housing_df['SalePrice'].describe()

In [ ]:
# Create function to reject outliers, default stdev is 1.5
def reject_outliers(data, m=1.5):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

# Box plot showing the outliers gone away with m=1.5
sns.boxplot(reject_outliers(housing_df['SalePrice']),
            orient='h').set_title('Removing Outliers with 1.5 Standard Deviations')
plt.show()

In [ ]:
# Make series of mask
mask = reject_outliers(housing_df['SalePrice']).tolist()
mask[0:20]

In [ ]:
# Mask as Boolean
mask_results = housing_df['SalePrice'].isin(mask)[0:20].tolist()
mask_results

In [ ]:
outliers_dict = dict(zip(mask, mask_results))
outliers_dict

In [ ]:
housing_df = housing_df[~housing_df['SalePrice'].isin(mask) == False]
housing_df.head(20)

In [2]:
housing_df['SalePrice'].describe()

NameError: name 'housing_df' is not defined

In [ ]:
# Columns with null (np.nan) values
nan_col_list = housing_df.columns[housing_df.isnull().any()].tolist()
nan_col_list

In [ ]:
nulls = housing_df[nan_col_list].isnull().sum()
nulls

In [ ]:
nulls.plot(kind='barh', title='Null Values per Column')
plt.show()

In [ ]:
print('Missing data in each dataframe column:')
for col in housing_df.columns:
    missing_data = len(housing_df) - housing_df[col].count()
    if (missing_data > 0 or missing_data =='NaN'):
         print(col, ':', missing_data, 'missing values is', str(round(float(missing_data / float(len(housing_df))) * 
             100, 3)), '% of total')

In [ ]:
col_na = ['Misc Feature',
          'Fence',
          'Pool QC',
          'Garage Cond',
          'Garage Qual',
          'Garage Finish',
          'Garage Type',
          'Fireplace Qu',
          'BsmtFin SF 1',
          'BsmtFin SF 2',
          'Bsmt Exposure',
          'Alley']

In [ ]:
housing_df[col_na] = housing_df[col_na].fillna('NA')
print(housing_df.shape)
housing_df.head()

In [ ]:
# Function to make random distribution based on existing data
def make_randoms(df, col):
    return np.random.normal(df[col].mean(),
                            scale=df[col].std(),
                            size=df[col].isnull().sum()).astype(int)

In [ ]:
# Use random number generator to create a series of numbers based on 'garage_yr_blt' statistics
garage_yr_blt_rand = make_randoms(housing_df,'Garage Yr Blt')
garage_yr_blt_rand

In [ ]:
# Plot the random distribution of the 'garage_yr_blt' column
plt.hist(garage_yr_blt_rand)
plt.xlabel('garage_yr_blt')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Use random number generator to create a series of numbers based on 'LotFrontage' statistics
lot_frontage_rand = make_randoms(housing_df,'Lot Frontage')
lot_frontage_rand

In [ ]:

# Plot the random distribution of the 'LotFrontage' column
plt.hist(lot_frontage_rand)
plt.xlabel('Lot Frontage')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Replace the NaNs with teh random data using this method because I don't
# have enough fucking time to find a more efficient method
housing_df['Lot Frontage'] = housing_df['Lot Frontage'].fillna('9999')
housing_df['Garage Yr Blt'] = housing_df['Garage Yr Blt'].fillna('9999')
housing_df.loc[housing_df['Lot Frontage']=='9999','Lot Frontage'] = lot_frontage_rand
housing_df.loc[housing_df['Garage Yr Blt']=='9999','Garage Yr Blt'] = garage_yr_blt_rand

# Drop the rest
print('Before: ', housing_df.shape)
housing_df = housing_df.dropna()
print('After: ', housing_df.shape)
Before:  (2673, 81)
After:  (2578, 81)

# Verify that all null values are gone
nan_col_list = housing_df.columns[housing_df.isnull().any()].tolist()
nan_col_list

nan_col_list = housing_df.columns[housing_df.isnull().any()].tolist()
nulls = housing_df[nan_col_list].isnull().sum()
nulls



housing_df.head(20)

In [ ]:
# Re-index housing_df so indices match 
housing_df.reset_index(inplace=True)
housing_df.head(20)
housing_df.drop('index', axis=1, inplace=True)

housing_df.drop('PID', axis=1, inplace=True)

In [ ]:
cat_columns = hous_df.select_dtypes(include=[object]).columns.tolist()
dummies_df = pd.get_dummies(housing_df[cat_columns],prefix_sep='_',drop_first=False)
print(dummies_df.shape)
dummies_df.head(20)